In [8]:
# Take magnitude as the feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Load dataset
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]
n_comp = 4

# Train-test split
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)

trainData - (800, 15, 251, 2)
testData - (200, 15, 251, 2)


In [10]:
# --------------------------------------------------- Preprocessing ---------------------------------------------------
def apply_pca(data, n_components):

    data_pca = []
    for cir in data:
        scaler = StandardScaler()
        cir_scaled = scaler.fit_transform(cir) # Shape: (251, 2)    
        
        pca = PCA(n_components=1)
        cir_pca_transformed = pca.fit_transform(cir_scaled)  # Shape: (251, n_components)
        
        data_pca.append(cir_pca_transformed)
        
    return data_pca
# ----------------- Train -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :] # (6400, 251, 2)
train_eve_cirs = trainCIR[:, eve_channel, :, :] # (6400, 251, 2)
train_cirs = np.vstack((train_alice_cirs, train_eve_cirs))


train_cirs_pca = apply_pca(train_cirs, n_components=1)
train_cirs_pca = np.array(train_cirs_pca)

print(train_cirs_pca.shape)  # (12800, 251, 1)
# train_cirs_pca = train_cirs_pca.reshape(train_cirs_pca.shape[0], -1)  # (12800, 251)

# # Labels
# alice_train_labels = np.zeros(alice_train_CIRs.shape[0])  # '0' for Alice
# eve_train_labels = np.ones(eve_train_CIRs.shape[0])       # '1' for Eve
# train_labels = np.hstack((alice_train_labels, eve_train_labels))  # (12800,)

# # ----------------- Test -----------------
# # Feature Extraction
# alice_test_CIRs = testCIR[:, alice_channel, :, :]
# eve_test_CIRs = testCIR[:, eve_channel, :, :]
# test_cirs = np.vstack((alice_test_CIRs, eve_test_CIRs))


# test_cirs_pca = []
# for cir in test_cirs:
#     # scaler = StandardScaler()
#     # cir_scaled = scaler.fit_transform(cir) # Shape: (251, 2)    
    
#     pca = PCA(n_components=n_components)
#     cir_pca_transformed = pca.fit_transform(cir)  # Shape: (251, n_components)
    
#     test_cirs_pca.append(cir_pca_transformed)

# test_cirs_pca = np.array(test_cirs_pca)
# test_cirs_pca = test_cirs_pca.reshape(test_cirs_pca.shape[0], -1)  # (3200, 251)
# # Labels
# alice_test_labels = np.zeros(alice_test_CIRs.shape[0])  # '0' for Alice
# eve_test_labels = np.ones(eve_test_CIRs.shape[0])       # '1' for Eve
# test_labels = np.hstack((alice_test_labels, eve_test_labels))  # (3200,)

(1600, 251, 1)


In [ ]:
# ----------------- Classification -----------------
# KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_cirs_pca, train_labels)

# Predict
predictions = knn.predict(test_cirs_pca)

In [ ]:
# ----------------- Evaluation -----------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Step 1: Create a dummy dataset (10 samples, 251 data points, 2 features)
data = np.random.randn(10, 251, 2)

data_pca = []
for cir in data:
    
    cir_reshaped = cir.T  # Shape: (2, 251)
    
    scaler = StandardScaler()
    cir_scaled = scaler.fit_transform(cir_reshaped.T).T  # Shape: (2, 251)
    
    # pca = PCA(n_components=2)
    # cir_pca_transformed = pca.fit_transform(cir_scaled.T)  # Shape: (251, 2)
    
    # cir_pca_transformed = cir_pca_transformed.T  # Shape: (2, 2)
    
    # data_pca.append(cir_pca_transformed)

data_pca = np.array(data_pca)  # Shape: (10, 2, 2)
print(data_pca.shape)  # Should output: (10, 2, 2)


(2, 251)
(251, 2)
(2, 251)

(0,)


In [4]:
import numpy as np

data = np.random.randn(10, 251, 2)
n_components = 3

# data_reduced = []
# for cir in data:
#     # cir has shape (251, 2)
#     # Transpose to (2, 251) to perform SVD along samples
#     cir_transposed = cir.T  # Shape: (2, 251)
    
#     # Perform SVD
#     U, s, Vt = np.linalg.svd(cir_transposed, full_matrices=False)
    
#     # Keep top n_components singular values/vectors
#     U_reduced = U[:, :n_components]  # Shape: (2, n_components)
#     s_reduced = s[:n_components]     # Shape: (n_components,)
#     Vt_reduced = Vt[:n_components, :]  # Shape: (n_components, 251)
    
#     # Reconstruct reduced data
#     cir_reduced = np.dot(U_reduced, np.diag(s_reduced))  # Shape: (2, n_components)
#     data_reduced.append(cir_reduced.T)  # Append with shape (n_components, 2)

# data_reduced = np.array(data_reduced)  # Shape: (10, 3, 2)
# print(data_reduced.shape)  # Outputs: (10, 3, 2)

data = np.random.randn(10, 251, 2)

# Separate real and imaginary parts
data_real = data[:, :, 0]  # Shape: (10, 251)
data_imag = data[:, :, 1]  # Shape: (10, 251)
print(data_real.shape)
print(data_imag.shape)

# Apply StandardScaler to standardize the data along the features axis
scaler_real = StandardScaler()
data_real_scaled = scaler_real.fit_transform(data_real)  # Shape: (10, 251)


scaler_imag = StandardScaler()
data_imag_scaled = scaler_imag.fit_transform(data_imag)  # Shape: (10, 251)


pca_components = 4
pca_real = PCA(n_components=pca_components)
data_real_pca = pca_real.fit_transform(data_real_scaled)  # Shape: (10, 4)
print(data_real_pca.shape)

pca_imag = PCA(n_components=pca_components)
data_imag_pca = pca_imag.fit_transform(data_imag_scaled)  # Shape: (10, 4)
print(data_imag_pca.shape)

data_pca = np.stack((data_real_pca, data_imag_pca), axis=2)  # Shape: (10, 4, 2)
print(data_pca.shape)


(10, 251)
(10, 251)
(10, 4)
(10, 4)
(10, 4, 2)
